# Purpose: Coloring Segmented Images According to Shape Mode

Date Created: 1-26-2021

Date Updated: 11-15-2021: Updated to change from the ferret paper to the Microglia/BeV paper

Author: Hawley Helmbrecht

**Colors:** 

shape mode 1: Blue

shape mode 2: Orange

shape mode 3: Green

shape mode 4: Red

shape mode 5: Purple

*Step 1: Import Necessary Packages*

In [ ]:
import numpy as np
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from PIL import Image
import os

*Step 2: User input variables*

In [ ]:
im_file_path = '/Users/hhelmbre/Desktop/Phuong_Bev/vampire/images/test/4_hr_exposure_0_hr_application_pngs/bev_treatment/EV_MICROGLIA_STUDY_4HR_EXPOSURE_1-2-1-1_li_thresh_quad2xy47c1.png'
csv_file_path = '/Users/hhelmbre/Desktop/Phuong_Bev/visualization/shape_mode_vs_feature_all.csv'

*Step 3: Reading in the Image and CSV with Labels*

In [ ]:
img = mpimg.imread(im_file_path)
csv_df = pd.read_csv(csv_file_path)

In [ ]:
csv_df

*Step 4: Getting the file name*

In [ ]:
im_file_name_split = im_file_path.split('/')
length = len(im_file_name_split)
file_name = im_file_name_split[length-1]
file_name

*Step 5: Creating a mask for labeling*

In [ ]:
mask_b = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)
mask_o = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)
mask_g = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)
mask_r = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)
mask_p = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)

In [ ]:
start_pt = (14,111)
cv2.floodFill(img, mask_o, start_pt, (0,0,255), flags=4)

In [ ]:
mask_o = mask_o[1:-1, 1:-1]

In [ ]:
plt.imshow(mask_o, cmap='gray')

*Step X: Automating the Process*

*Step 1: Import need_to_label_list*

In [ ]:
need_to_label_list_file_path = '/Users/hhelmbre/Desktop/Phuong_Bev/visualization/images_to_color.csv'
need_to_label_list = pd.read_csv(need_to_label_list_file_path)
need_to_label_list

In [ ]:
need_to_label_list2 = []
j = 0
for file_paths in range(0,need_to_label_list.shape[0]):
    arr = os.listdir(need_to_label_list['image_file_path'][j])
    image_list = np.asarray(arr)
    image_list = [x for x in image_list if ".png" in x]
    for images in image_list:
        file_name = str(need_to_label_list['image_file_path'][j] + '/' + images)
        need_to_label_list2.append(file_name)
        
    j += 1
    

*Add in a step here to visualize the shape mode labels with the colors I chose*

In [ ]:
def c(x):
   col = plt.cm.twilight(x)
   fig, ax = plt.subplots(figsize=(1,1))
   fig.set_facecolor(col)
   ax.axis("off")
   plt.show()

In [ ]:
color_list = np.linspace(0,1,20)
color_list = [0.5, 0.047, 0.7, 0.33, 0.9]
for options in color_list:
    print(options)
    c(options)

*The code to actually recolor the images*

In [ ]:
label_save_file_path = '/Users/hhelmbre/Desktop/Phuong_Bev/vampire/color_coded'

In [ ]:
k = 0
cmap = plt.get_cmap('twilight')
cmap.set_bad(color = 'white')
for images in need_to_label_list2:
    
    #Reading in the image and its labels
    im_file_path = images
    img = mpimg.imread(im_file_path)
    csv_df = pd.read_csv('/Users/hhelmbre/Desktop/Phuong_Bev/visualization/shape_mode_vs_feature_all.csv')
    
    #Getting the file path from the file name
    im_file_name_split = im_file_path.split('/')
    length = len(im_file_name_split)
    file_name = im_file_name_split[length-1]
    print(file_name)
    
    mask = np.zeros(np.asarray(img.shape)+2, dtype=np.uint8)
    
    csv_df = csv_df[csv_df.Filename == file_name]
    csv_df = csv_df.reset_index(drop=True)
    
    for shapes in range(len(csv_df)):
        shape_mode = (csv_df['Shape mode'][shapes]).astype(int)
        start_pt = (csv_df['X'][shapes], csv_df['Y'][shapes])
        area =csv_df['Area'][shapes]
        
        if img[csv_df['Y'][shapes]][csv_df['X'][shapes]]== 1:
            if shape_mode==1: #0.5
                cv2.floodFill(img, mask, start_pt, 0, flags=0)
                mask[mask == 1] = 128

            elif shape_mode==2: #0.047
                cv2.floodFill(img, mask, start_pt, 0, flags=0)
                mask[mask == 1] = 12
            elif shape_mode==3: #0.7
                cv2.floodFill(img, mask, start_pt, 0, flags=0)
                mask[mask == 1] = 179

            elif shape_mode==4: #0.33
                cv2.floodFill(img, mask, start_pt, 0, flags=0)
                mask[mask == 1] = 85

            elif shape_mode==5: #0.9
                cv2.floodFill(img, mask, start_pt, 0, flags=0)
                mask[mask == 1] = 230

            if shapes == len(csv_df)-1:
                mask = mask[1:-1, 1:-1]
                mask[0][0] = 255
                
                mask = mask.astype('float')
                mask[mask==0] = np.nan
            
                plt.imshow(mask,cmap='twilight')
                plt.tick_params(
                    axis='x',
                    which='both',
                    bottom=False,
                    top=False,
                    labelbottom=False)
                plt.yticks([])
                plt.savefig(str(label_save_file_path + str(k) + file_name), bbox_inches = 'tight',
    pad_inches = 0)

                #img_to_save = Image.fromarray(mask)
                #img_to_save.save(str(label_save_file_path + str(k) + file_name))

    
    k += 1

*Step 3: Environment Specs*

In [ ]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,cv2,PIL,wget

%watermark -u -n -t -z